In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk

from datetime import datetime, timedelta 

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as ex
import plotly.io as pio

import glob
import os

import seaborn
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV

from joblib import dump, load
 
import tensorflow 
import keras as ks
from keras.models import Sequential
from keras.layers import Dense, Input 
from keras.activations import relu, tanh, linear
from keras.models import load_model

from keras.callbacks import EarlyStopping
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.utils import shuffle
from sklearn.metrics import make_scorer

In [ ]:
from readIn import readIn
from removeT_bett import removeT_bett

1. Relevante Parameter und Einstellungen definieren

In [ ]:
# Data directory
datadir = "X:\KI Praktikum\\validate_Data\ipo_time"

# Sollen die Daten interpoliert werden?
do_interpolation = False
# Sollen die interpolierten csv Dateien extra gespeichert werden? Und der gesamte df?
safeSingleCsv = False
saveDfCsv = False

# Modell directory
pipelinedir = "X:\\KI Praktikum\\pipeline.p"
modelldir = "X:\\KI Praktikum\\validate_Data\\models\\keras_NN_321_Nadam.h5"

# Aufteilung der trainings, validation und test Daten:
split_evenly = True
split_by_coolant = False
split_by_profile = False

2. Die notwendigen Daten einlesen

In [ ]:
df_merged = readIn(datadir, ipo=do_interpolation, safeSingleCsv = safeSingleCsv)
if saveDfCsv:
    df_merged.to_csv("X:\\KI Praktikum\\validate_Data\\merged_df.csv")

# z_ipo skalieren
df_merged["welle_z_ipo_time"] = 1000* df_merged["welle_z_ipo_time"]


In [ ]:
if split_evenly:
    validation_files = ['X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch10_12_2022_trocken_iso_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch22_082022_trocken_iso_5s_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch18_11_2022_M8_iso_shifted_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch17_01_2023_M127_iso__time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch18_01_2023_M8_M127_iso__time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch24_092022_trocken_prof1_5s_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch22_092022_trocken_prof3_5s_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch18_01_2023_M121_iso__time_ipo.csv'
                        ]
    test_files = ['X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch24_11_2022_trocken_iso_time_ipo.csv',
                'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch12_11_2022_M8_iso__time_ipo.csv',
                'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch19_11_2022_M8_prof2_time_ipo.csv',
                'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch21_092022_trocken_prof2_5s_time_ipo.csv',
                'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch24_092022_trocken_prof2_5s_time_ipo.csv']

if split_by_coolant:
    validation_files = ['X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch11_12_2022_M8_iso_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch12_11_2022_M127_iso_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch12_11_2022_M8_iso__time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch13_01_2023_M127_iso_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch15_11_2022_M7_iso_shifted_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch17_01_2023_M121_iso__time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch17_01_2023_M127_iso__time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch18_01_2023_M121_iso__time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch18_01_2023_M8_M127_iso__time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch18_082022_M8_iso_5s_time_ipo.csv']
    test_files = ['X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch18_11_2022_M8_iso_shifted_time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch19_01_2023_M8_M121_isoteil_time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch19_11_2022_M8_prof2_time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch19_11_2022_M8_prof4_time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch2023_01_12_14_10_M8_iso__time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch2023_01_13_09_30_M7_iso__time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch2023_01_17_17_04_M127_iso__time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch2023_01_18_06_00_M121_iso__time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch20_11_2022_M8_prof3_time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch24_11_2022_M8_iso_time_ipo.csv']
if split_by_profile:
    validation_files = ['X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch19_11_2022_M8_prof2_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch19_11_2022_M8_prof4_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch20_092022_trocken_prof1_5s_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch20_11_2022_M8_prof3_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch21_092022_trocken_prof1_5s_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch21_092022_trocken_prof2_5s_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch18_01_2023_M8_M127_iso__time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch24_11_2022_trocken_iso_time_ipo.csv']
    test_files = ['X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch22_092022_trocken_prof3_5s_time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch24_092022_trocken_prof1_5s_time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch24_092022_trocken_prof2_5s_time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch25_092022_trocken_prof3_5s_time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch17_01_2023_M127_iso__time_ipo.csv']
    
df_train = df_merged[df_merged["filename"].isin(validation_files) == False]
df_train = df_train[df_train["filename"].isin(test_files) == False]
df_validation = df_merged[df_merged["filename"].isin(validation_files) == True]
df_test = df_merged[df_merged["filename"].isin(test_files) == True]

In [ ]:
y_train = df_train["welle_z_ipo_time"]
parameters = ["t_bett", "t_motor", "t_spindle", "M8", "M121", "M127", "M7"]
x_train = df_train[parameters].to_numpy()

y_validation = df_validation["welle_z_ipo_time"]
x_validation = df_validation[parameters].to_numpy()

y_test = df_test["welle_z_ipo_time"]
x_test = df_test[parameters].to_numpy()

3. Ein gespeichertes Modell einlesen

In [ ]:
modell = load_model(modelldir)

4. Train the model and compute the deviations

In [ ]:
pipeline = load(pipelinedir)
x_transform = pipeline.fit_transform(x_train)
x_validation = pipeline.transform(x_validation)
x_test = pipeline.transform(x_test)

In [ ]:
summary = modell.fit(x_transform, y_train)

In [ ]:
#score = summary.score(x_validation, y_validation)
pred = modell.predict(x_validation)
error_avg = np.mean(abs(y_validation - pred.flatten()))
error_max = max(abs(y_validation - pred.flatten()))
print("max: ", error_max)
print("avg: ", error_avg)